# TRATAMENTO DE DADOS

In [3]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import *

In [4]:
# UPAR O DATA SET E CHECAR O DIRETORIO DOS ARQUIVOS PARA NÃO DAR ERRO

df_2 = pd.read_excel('/content/2_detail_table_customers.xls') #Table customers
df_3 = pd.read_excel('/content/3_detail_table_vehicles.xls') #Table vehicles
df_4 = pd.read_excel('/content/4_detail_table_depots.xls') #Table depots
df_5 = pd.read_excel('/content/5_detail_table_constraints_sdvrp.xls') #Table constraints
df_6 = pd.read_excel('/content/6_detail_table_cust_depots_distances.xls') #Table cust

df_list = [df_2,df_3,df_4,df_5,df_6] #Lista com todos os dataframes
#print(len(df_list))


In [5]:
ROUTES_IDS = df_2['ROUTE_ID'].unique() #Cria vetor com o ID de cada set de rotas

id_cliente_desejado = ROUTES_IDS[0] #Começaremos pegando o primeiro ID

'''
df_route1 = df_2.loc[df_2['ROUTE_ID'] == ROUTES_IDS[0]] #Filtra pelo primeiro ID
df_depot1 = df_4.loc[df_4['ROUTE_ID'] == ROUTES_IDS[0]] #filtra a tabela 4 pelo ID
df_route1.head()
'''


"\ndf_route1 = df_2.loc[df_2['ROUTE_ID'] == ROUTES_IDS[0]] #Filtra pelo primeiro ID\ndf_depot1 = df_4.loc[df_4['ROUTE_ID'] == ROUTES_IDS[0]] #filtra a tabela 4 pelo ID\ndf_route1.head()\n"

In [6]:
# Função que recebe ID da rota e a matriz de clientes
      # Retorna: vetor com tuplas, as quais contem limite de peso/volume/tempo_descarrregamento

def limits_route(ROUTE_ID, df):

  df_route1 = df.loc[df['ROUTE_ID'] == ROUTE_ID] #Filtra pelo primeiro ID

  tab_limites = ['CUSTOMER_CODE','TOTAL_WEIGHT_KG','TOTAL_VOLUME_M3','CUSTOMER_DELIVERY_SERVICE_TIME_MIN']

  df_lim = df_route1[tab_limites].copy() #Tabela com os requisitos de peso, volume e tempo gasto na entrega de cada cliente

  lim_peso = df_lim['TOTAL_WEIGHT_KG'].tolist()
  lim_volume = df_lim['TOTAL_VOLUME_M3'].tolist()
  lim_time = df_lim['CUSTOMER_DELIVERY_SERVICE_TIME_MIN'].tolist()

  lista_limites = []

  for i in range(len(lim_peso)):
    lista_limites.append([lim_peso[i], lim_volume[i], lim_time[i]])

  return lista_limites

# Teste da função

lista = limits_route(ROUTES_IDS[0],df_2)
print (lista)

[[64.318, 0.32, 19.94], [17.553, 0.046, 11.36], [6.378, 0.041, 8.73], [16.375, 0.046, 9.1], [18.939, 0.074, 11.99], [25.334, 0.097, 10.64], [582.802, 5.0, 53.03], [364.289, 3.0, 18.18], [109.573, 0.308, 11.08], [24.46, 0.061, 10.49], [5.957, 0.039, 8.22], [12.978, 0.025, 10.48], [32.848, 0.11, 15.27], [4.826, 0.061, 8.05], [9.289, 0.024, 7.94], [25.956, 0.058, 8.24], [172.633, 2.0, 21.57], [15.75, 0.028, 8.02], [30.806, 0.073, 11.35], [6.474, 0.029, 8.93], [6.294, 0.013, 7.87], [20.065626, 0.042516, 9.12], [6.859629, 0.030696, 9.19], [15.429162, 0.048828, 10.35], [34.608, 0.077616, 8.4], [7.9128, 0.051564, 8.29], [16.904937, 0.068292, 9.45], [74.042, 0.217, 9.61], [33.29655, 0.084984, 11.92], [25.24725, 0.078468, 13.07], [0.617, 0.004, 7.77], [36.1935, 0.07518, 11.16], [15.4917, 0.062796, 10.16], [21.1785, 0.066276, 11.98], [311.472, 1.0, 13.63], [17.8185, 0.072456, 9.47], [12.778248, 0.0552, 9.6], [54.601, 0.165, 12.21], [16.6022325, 0.085212, 9.51], [17.72295, 0.042192, 9.34], [33.49

In [7]:
# Função que recebe ID da rota, matriz de clientes e matriz de depots
      # Retorna: matriz de distancias entre clientes e clientes e clientes e deposito
      # O deposito está indicado com indice 0 (zero) na matriz

def distance(ROUTE_ID, df1, df2):
  df_route = df1.loc[df1['ROUTE_ID'] == ROUTE_ID] #Filtra a tabela de clientes pelo primeiro ID
  df_depot = df2.loc[df2['ROUTE_ID'] == ROUTE_ID] #filtra a tabela de depot pelo ID
  #Começa o tratamento de dados do dataframe

  tab_dist = ['CUSTOMER_LATITUDE','CUSTOMER_LONGITUDE']
  tab_depot = ['DEPOT_LATITUDE','DEPOT_LONGITUDE']

  df_dist = df_route[tab_dist].copy()
  df_depot = df_depot[tab_depot].copy()

  df_dist.columns,df_depot.columns  = ['LATITUDE', 'LONGITUDE'],['LATITUDE', 'LONGITUDE']

  latitude_list = df_dist['LATITUDE'].tolist()
  latitude_list.append(df_depot['LATITUDE'].tolist()[0])

  longitude_list = df_dist['LONGITUDE'].tolist()
  longitude_list.append(df_depot['LONGITUDE'].tolist()[0])

  #Desliza 1 unidade para direita, para que o deposito fique na primeira posição

  latitude_list = latitude_list[-1:] + latitude_list[:-1]
  longitude_list = longitude_list[-1:] + longitude_list[:-1]

  #Termina o tratamento de dados do dataframe e cria um novo

  new_df = pd.DataFrame({'LATITUDE': latitude_list, 'LONGITUDE': longitude_list})

  distance_matrix = []

  for i in range(len(latitude_list)):
    distance_matrix.append([])
    for j in range(len(longitude_list)):
      distance_matrix[i].append(math.sqrt((longitude_list[i] - longitude_list[j]) ** 2 +
                                    (latitude_list[i] - latitude_list[j]) ** 2))
  return distance_matrix

distancias = distance(ROUTES_IDS[0],df_2,df_4)
print(distancias)

[[0.0, 0.2769414601804545, 0.29544950825842564, 0.28128030831990153, 0.5045318830653654, 0.46203545374192406, 0.30510012996971275, 0.20383444917354676, 0.32002378729500885, 0.1982473475153774, 0.27056169473050673, 0.5041942615806718, 0.28186800779351423, 0.504046193495874, 0.24781085353008794, 0.29420989528964137, 0.19398048855408936, 0.2802779454986132, 0.20568055249411288, 0.22008651612373695, 0.22570727150112554, 0.1763468168982505, 0.5038797866778518, 0.28498702640482965, 0.21669871574944516, 0.19055780668469852, 0.49697558995053515, 0.17365583902974255, 0.2848643223839793, 0.370513971242125, 0.5086777489335284, 0.5049761228672888, 0.5096072498668743, 0.39486367117358734, 0.5140645261854582, 0.29139947045505044, 0.28226035548164785, 0.25027157752553675, 0.31831536449708075, 0.2917689573160043, 0.29591562807848293, 0.46053411514879644, 0.45573054521930695, 0.5039953324953337, 0.2824422551597, 0.20133300964989037, 0.1935523404066281, 0.29250669540419466, 0.19929796758778132, 0.332234